# Summerize Shareholding:

#### Summerize: 

- For all stocks from RawReports folder, read specific Excels sheet into dataframe  
- Add stock name column to df 
- Transform the df 
- Consolidate the df 

#### Analyze:

- Compute something
- Write the computed new data 
- Analyze or visualize the new data 

#### Summary Report File Name Convention: 
- Summary01-Top-Ratios 
- Summary02-Quarterly-Results
- Summary03-Profit-And-Loss
- Summary04-Compounded-Sales-Growth
- Summary05-Compounded-Profit-Growth
- Summary06-Stock-Price-CAGR 
- Summary07-Return-on-Equity
- Summary08-Balance-Sheet
- Summary09-Cash-Flows
- Summary10-Ratios
- Summary11-Shareholding-Pattern

## 1.1 Imports:

In [1]:
import requests as rq
import pandas as pd 
import time
import datetime
from bs4 import BeautifulSoup
import re
import os


## 1.2 Define Inputs:

>  ⚠️ <span style="color:red"> **Update the CSV File below to read as needed:**</span>

In [5]:
## CAREFULLY UPDATE THIS. This should be same as what specified in RawReport notebook

# For My Watchlisted stocks
input_cvs_file = "../Input/" + "stocks-watchlisted.csv"
# For PSU stocks
input_cvs_file = "../Input/" + "stocks-psu.csv"

input_cvs_file = "../Input/" + "stocks-watchlisted.csv"

## UPDATE THIS. The folder is in  ../Output/RawReports-01/. This is where raw reports are read and summerized

raw_report_folder_name     = "20240817-132127"


In [6]:
raw_report_folder_path = "../Output/RawReports-01/" + raw_report_folder_name

df = pd.read_csv(input_cvs_file)
df.head(5)


,Screener Stock Symbol,Url Segment
0,526433,consolidated
1,532407,consolidated
2,544021,consolidated
3,AARTIDRUGS,consolidated
4,AARTIIND,consolidated


## 1.3 Process reports:

In [7]:
df_profit_n_loss = pd.DataFrame([])

for index, row in df.iterrows():
    stock_symbol = row["Screener Stock Symbol"]
    filepath = raw_report_folder_path + "/" + stock_symbol + ".xlsx"
    df_data =  pd.read_excel(filepath, sheet_name="Profit & Loss")
    print("Processing stock ",stock_symbol)
    # Add stockname column and move into 2nd position: 
    df_data['Stockname'] = stock_symbol
    column_to_move = df_data.pop("Stockname")
    df_data.insert(1, "Stockname", column_to_move)
    df_profit_n_loss = pd.concat([df_profit_n_loss, df_data], ignore_index=True, sort=False)
    
print("All done!")    

Processing stock  526433
Processing stock  532407
Processing stock  544021
Processing stock  AARTIDRUGS
Processing stock  AARTIIND
Processing stock  AARTIPHARM
Processing stock  AARTISURF
Processing stock  ABB
Processing stock  ACC
Processing stock  ADANIENT
Processing stock  AFFLE
Processing stock  ALKYLAMINE
Processing stock  ANGELONE
Processing stock  ARE&M
Processing stock  ASAHIINDIA
Processing stock  ASHOKLEY
Processing stock  ASIANPAINT
Processing stock  ASTRAL
Processing stock  AXISBANK
Processing stock  BAJAJ-AUTO
Processing stock  BAJAJELEC
Processing stock  BAJAJFINSV
Processing stock  BAJFINANCE
Processing stock  BALAMINES
Processing stock  BATAINDIA
Processing stock  BEL
Processing stock  BERGEPAINT
Processing stock  BHARTIARTL
Processing stock  BOROLTD
Processing stock  BRITANNIA
Processing stock  BSE
Processing stock  CAMS
Processing stock  CAMPUS
Processing stock  CDSL
Processing stock  CENTUM
Processing stock  CGPOWER
Processing stock  COCHINSHIP
Processing stock  COFO

## 1.5 Inspect dataframe:

In [8]:
print(len(df_profit_n_loss))

1920


In [9]:
df_profit_n_loss.describe()

,Profit and Loss,Stockname,Mar 2013,Mar 2014,Mar 2015,Mar 2016,Mar 2017,Mar 2018,Mar 2019,Mar 2020,...,Sep 2015,Sep 2016,Sep 2017,Sep 2018,Sep 2019,Sep 2020,Sep 2021,Sep 2022,Sep 2023,Mar 2012
count,1920,1920,1068,1139,1296,1376,1415,1497,1690,1763,...,12,12,12,12,12,12,12,12,12,12
unique,15,160,675,717,795,826,865,933,989,1049,...,11,12,12,12,12,12,11,11,12,12
top,Expenses,526433,1,0,31%,0,0,0%,0%,0%,...,30%,10837,11065,12795,13084,9946,1452,23%,19554,102
freq,160,12,15,20,15,25,18,24,30,36,...,2,1,1,1,1,1,2,2,1,1


In [10]:
df_profit_n_loss.columns

Index(['Profit and Loss', 'Stockname', 'Mar 2013', 'Mar 2014', 'Mar 2015',
       'Mar 2016', 'Mar 2017', 'Mar 2018', 'Mar 2019', 'Mar 2020', 'Mar 2021',
       'Mar 2022', 'Mar 2023', 'Mar 2024', 'TTM', 'Mar 2010', 'Mar 2011',
       'Dec 2012', 'Dec 2013', 'Dec 2014', 'Dec 2015', 'Dec 2016', 'Dec 2017',
       'Dec 2018', 'Dec 2019', 'Dec 2020', 'Dec 2021', 'Dec 2022', 'Dec 2023',
       'Jun 2013', 'Jun 2014', 'Jun 2015', 'Sep 2010', 'Sep 2011', 'Sep 2014',
       'Sep 2015', 'Sep 2016', 'Sep 2017', 'Sep 2018', 'Sep 2019', 'Sep 2020',
       'Sep 2021', 'Sep 2022', 'Sep 2023', 'Mar 2012'],
      dtype='object')

## 1.6 Sort column ordering:

In [11]:
date = '1970-01-01'
df_profit_n_loss = df_profit_n_loss.sort_index(
    axis=1, 
    key=lambda x: pd.to_datetime(x, errors='coerce', format='mixed').fillna(date)
)

In [12]:
df_profit_n_loss.columns

Index(['Profit and Loss', 'Stockname', 'TTM', 'Mar 2010', 'Sep 2010',
       'Mar 2011', 'Sep 2011', 'Mar 2012', 'Dec 2012', 'Mar 2013', 'Jun 2013',
       'Dec 2013', 'Mar 2014', 'Jun 2014', 'Sep 2014', 'Dec 2014', 'Mar 2015',
       'Jun 2015', 'Sep 2015', 'Dec 2015', 'Mar 2016', 'Sep 2016', 'Dec 2016',
       'Mar 2017', 'Sep 2017', 'Dec 2017', 'Mar 2018', 'Sep 2018', 'Dec 2018',
       'Mar 2019', 'Sep 2019', 'Dec 2019', 'Mar 2020', 'Sep 2020', 'Dec 2020',
       'Mar 2021', 'Sep 2021', 'Dec 2021', 'Mar 2022', 'Sep 2022', 'Dec 2022',
       'Mar 2023', 'Sep 2023', 'Dec 2023', 'Mar 2024'],
      dtype='object')

In [13]:
df_profit_n_loss.head(5)

,Profit and Loss,Stockname,TTM,Mar 2010,Sep 2010,Mar 2011,Sep 2011,Mar 2012,Dec 2012,Mar 2013,...,Mar 2021,Sep 2021,Dec 2021,Mar 2022,Sep 2022,Dec 2022,Mar 2023,Sep 2023,Dec 2023,Mar 2024
0,Sales,526433,203,NaN,NaN,NaN,NaN,NaN,NaN,175,...,137,NaN,NaN,192,NaN,NaN,220,NaN,NaN,202
1,Expenses,526433,198,NaN,NaN,NaN,NaN,NaN,NaN,157,...,120,NaN,NaN,170,NaN,NaN,200,NaN,NaN,196
2,Operating Profit,526433,6,NaN,NaN,NaN,NaN,NaN,NaN,18,...,17,NaN,NaN,21,NaN,NaN,21,NaN,NaN,6
3,OPM %,526433,3%,NaN,NaN,NaN,NaN,NaN,NaN,10%,...,12%,NaN,NaN,11%,NaN,NaN,9%,NaN,NaN,3%
4,Other Income,526433,8,NaN,NaN,NaN,NaN,NaN,NaN,2,...,3,NaN,NaN,7,NaN,NaN,6,NaN,NaN,5


## 1.7 Save summary report:

In [14]:
output_path = "../Output/SummaryReports/"+"Summary03-Profit-And-Loss-"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+".csv"

df_profit_n_loss.to_csv(output_path,index=False,sep=",")